In [1]:
import datetime as dt
import numpy as np
import pandas as pd
import requests
import json
from pprint import pprint
import csv
# from api_key import api_key
from flask_sqlalchemy import SQLAlchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine

C:\Users\Bashira\Anaconda3\lib\site-packages\ipykernel\parentpoller.py:116: UserWarning: Parent poll failed.  If the frontend dies,
                the kernel may be left running.  Please let us know
                about your system (bitness, Python, etc.) at
                ipython-dev@scipy.org
  ipython-dev@scipy.org""")


In [2]:
export_url = (f"https://api.census.gov/data/timeseries/intltrade/exports")
import_url = (f"https://api.census.gov/data/timeseries/intltrade/imports")
china_code = "5700"
usa_code = "1000"
month = []
year = []

In [3]:
export_by_commodity = requests.get(f"{export_url}/hs?get=E_COMMODITY,E_COMMODITY_SDESC,ALL_VAL_MO,ALL_VAL_YR&time=from+2015-01&COMM_LVL=HS2&CTY_CODE={china_code}").json()


In [95]:
export_by_commodity

[['E_COMMODITY',
  'E_COMMODITY_SDESC',
  'ALL_VAL_MO',
  'ALL_VAL_YR',
  'time',
  'COMM_LVL',
  'CTY_CODE'],
 ['01', 'LIVE ANIMALS', '772193', '772193', '2015-01', 'HS2', '5700'],
 ['01', 'LIVE ANIMALS', '2735295', '3507488', '2015-02', 'HS2', '5700'],
 ['01', 'LIVE ANIMALS', '575410', '4082898', '2015-03', 'HS2', '5700'],
 ['01', 'LIVE ANIMALS', '513303', '4596201', '2015-04', 'HS2', '5700'],
 ['01', 'LIVE ANIMALS', '412544', '5008745', '2015-05', 'HS2', '5700'],
 ['01', 'LIVE ANIMALS', '891377', '5900122', '2015-06', 'HS2', '5700'],
 ['01', 'LIVE ANIMALS', '886743', '6786865', '2015-07', 'HS2', '5700'],
 ['01', 'LIVE ANIMALS', '1644213', '8431078', '2015-08', 'HS2', '5700'],
 ['01', 'LIVE ANIMALS', '863735', '9294813', '2015-09', 'HS2', '5700'],
 ['01', 'LIVE ANIMALS', '398464', '9693277', '2015-10', 'HS2', '5700'],
 ['01', 'LIVE ANIMALS', '130824', '9824101', '2015-11', 'HS2', '5700'],
 ['01', 'LIVE ANIMALS', '420284', '10244385', '2015-12', 'HS2', '5700'],
 ['01', 'LIVE ANIMALS',

In [87]:
count = 1
movalue = []
yrvalue = []
period = []
desc = []
code = []
while count < 4745:
    count += 1
    movalue.append(export_by_commodity[count][2])
    yrvalue.append(export_by_commodity[count][3])
    desc.append(export_by_commodity[count][1])
    period.append(export_by_commodity[count][4])
    code.append(export_by_commodity[count][0])
china_data_export = pd.DataFrame({"YTD Value" : yrvalue,
                           "Monthly Value" : movalue,
                          "Period" : period,
                          "Commodity" : desc,
                          "Comm Code": code})

In [6]:
import_by_commodity = requests.get(f"{import_url}/hs?get=I_COMMODITY,I_COMMODITY_SDESC,GEN_VAL_MO,GEN_VAL_YR&time=from+2015-01&COMM_LVL=HS2&CTY_CODE={china_code}").json()


In [7]:
len(import_by_commodity)

4803

In [83]:
count = 1
movalue = []
yrvalue = []
period = []
desc = []
code = []
while count < 4802:
    count += 1
    movalue.append(import_by_commodity[count][2])
    yrvalue.append(import_by_commodity[count][3])
    desc.append(import_by_commodity[count][1])
    period.append(import_by_commodity[count][4])
    code.append(import_by_commodity[count][0])
china_data_import = pd.DataFrame({"YTD Value" : yrvalue,
                           "Monthly Value" : movalue,
                          "Period" : period,
                          "Commodity" : desc,
                          "Comm Code": code})

In [102]:
china_data_export["YTD Value"] =pd.to_numeric(china_data_export["YTD Value"])
china_data_import["YTD Value"] =pd.to_numeric(china_data_import["YTD Value"])
china_data_import["Monthly Value"] =pd.to_numeric(china_data_import["Monthly Value"])
china_data_export["Monthly Value"] =pd.to_numeric(china_data_export["Monthly Value"])

In [106]:
monthly_2015 = china_data_export.loc[china_data_export['Period'].str.contains("2015")]

In [107]:
monthly_2015.groupby(["Period"])["Monthly Value"].sum()

Period
2015-01     9458426201
2015-02     8754588178
2015-03     9886519164
2015-04     9279912850
2015-05     8749787563
2015-06     9615754115
2015-07     9505173559
2015-08     9183456081
2015-09     9419421166
2015-10    11324856248
2015-11    10603592681
2015-12    10091105315
Name: Monthly Value, dtype: int64

In [11]:
def yearly_data(trade,year):
    data = trade[trade["Period"].str.contains(year)]
    data = data.groupby(["Comm Code","Commodity"])["YTD Value"].max()
    data = pd.DataFrame({"total" : data})
#     data = data.nlargest(10,"total")
    data = data.reset_index()
    
    return data
    
    

In [12]:
# first_2015 = china_data_export[china_data_export["Period"].str.contains("2015")]

In [13]:
# data_2015 = first_2015.groupby(["Comm Code","Commodity"])["YTD Value"].sum()
china_data_export["YTD Value"].sum()

3041011721625

In [14]:
# test= pd.DataFrame({"total" : data_2015})
china_data_export["Commodity"].nunique()

97

In [15]:
china_data_import["YTD Value"].sum()

12491142219444

In [16]:
data_2015_import = yearly_data(china_data_import,"2015")
data_2015_export = yearly_data(china_data_export,"2015")

data_2016_import = yearly_data(china_data_import,"2016")
data_2016_export = yearly_data(china_data_export,"2016")

data_2017_import = yearly_data(china_data_import,"2017")
data_2017_export = yearly_data(china_data_export,"2017")

data_2018_import = yearly_data(china_data_import,"2018")
data_2018_export = yearly_data(china_data_export,"2018")


In [76]:
data_2018_export

,Comm Code,Commodity,total
0,01,LIVE ANIMALS,14668230
1,02,MEAT AND EDIBLE MEAT OFFAL,391679816
2,03,"FISH, CRUSTACEANS & AQUATIC INVERTEBRATES",1064907299
3,04,DAIRY PRODS; BIRDS EGGS; HONEY; ED ANIMAL PR N...,315926940
4,05,"PRODUCTS OF ANIMAL ORIGIN, NESOI",317830044
5,06,"LIVE TREES, PLANTS, BULBS ETC.; CUT FLOWERS ETC.",4578705
6,07,EDIBLE VEGETABLES & CERTAIN ROOTS & TUBERS,33830616
7,08,EDIBLE FRUIT & NUTS; CITRUS FRUIT OR MELON PEEL,451676290
8,09,"COFFEE, TEA, MATE & SPICES",29505935
9,10,CEREALS,698031515


In [18]:
change_15_to_16 = (data_2016_export["total"]-data_2015_export["total"])/data_2015_export["total"]
change_16_to_17 = (data_2017_export["total"]-data_2016_export["total"])/data_2016_export["total"]
change_17_to_18 = (data_2018_export["total"]-data_2017_export["total"])/data_2017_export["total"]
# pd.set_option('display.long_format', lamba d: '')
df = pd.DataFrame({"code" : data_2015_export["Comm Code"],
                           "commodity" :data_2015_export["Commodity"] ,
                           "total 2015" : data_2015_export["total"],
                           "Change 15 to 16" : change_15_to_16,
                           "total 2016" : data_2016_export["total"],
                           "Change 16 to 17": change_16_to_17,
                           "total 2017": data_2017_export["total"],
                            "Change 17 to 18": change_17_to_18,
                            "total 2018": data_2018_export["total"]
                          })
df
# test_df_export = df[df["total 2018"] > 1000000000]

,code,commodity,total 2015,Change 15 to 16,total 2016,Change 16 to 17,total 2017,Change 17 to 18,total 2018
0,01,LIVE ANIMALS,10244385,-0.375314,6399523,1.614141,16729257,-0.123199,14668230
1,02,MEAT AND EDIBLE MEAT OFFAL,335616368,0.741901,584610550,-0.107270,521899634,-0.249511,391679816
2,03,"FISH, CRUSTACEANS & AQUATIC INVERTEBRATES",1007942885,-0.058282,949197817,0.301720,1235589412,-0.138138,1064907299
3,04,DAIRY PRODS; BIRDS EGGS; HONEY; ED ANIMAL PR N...,293604977,-0.093284,266216245,0.536488,409038171,-0.227635,315926940
4,05,"PRODUCTS OF ANIMAL ORIGIN, NESOI",266200493,-0.045513,254085015,0.060236,269390023,0.179814,317830044
5,06,"LIVE TREES, PLANTS, BULBS ETC.; CUT FLOWERS ETC.",7269741,-0.247339,5471654,-0.181904,4476338,0.022868,4578705
6,07,EDIBLE VEGETABLES & CERTAIN ROOTS & TUBERS,49126621,-0.183282,40122600,0.185720,47574169,-0.288887,33830616
7,08,EDIBLE FRUIT & NUTS; CITRUS FRUIT OR MELON PEEL,350212968,0.084048,379647558,0.286638,488468993,-0.075322,451676290
8,09,"COFFEE, TEA, MATE & SPICES",13702701,0.270346,17407167,0.275173,22197144,0.329267,29505935
9,10,CEREALS,2466361238,-0.475434,1293769326,0.042660,1348961712,-0.482542,698031515


In [19]:
# test_df_export.round(2)
df

,code,commodity,total 2015,Change 15 to 16,total 2016,Change 16 to 17,total 2017,Change 17 to 18,total 2018
0,01,LIVE ANIMALS,10244385,-0.375314,6399523,1.614141,16729257,-0.123199,14668230
1,02,MEAT AND EDIBLE MEAT OFFAL,335616368,0.741901,584610550,-0.107270,521899634,-0.249511,391679816
2,03,"FISH, CRUSTACEANS & AQUATIC INVERTEBRATES",1007942885,-0.058282,949197817,0.301720,1235589412,-0.138138,1064907299
3,04,DAIRY PRODS; BIRDS EGGS; HONEY; ED ANIMAL PR N...,293604977,-0.093284,266216245,0.536488,409038171,-0.227635,315926940
4,05,"PRODUCTS OF ANIMAL ORIGIN, NESOI",266200493,-0.045513,254085015,0.060236,269390023,0.179814,317830044
5,06,"LIVE TREES, PLANTS, BULBS ETC.; CUT FLOWERS ETC.",7269741,-0.247339,5471654,-0.181904,4476338,0.022868,4578705
6,07,EDIBLE VEGETABLES & CERTAIN ROOTS & TUBERS,49126621,-0.183282,40122600,0.185720,47574169,-0.288887,33830616
7,08,EDIBLE FRUIT & NUTS; CITRUS FRUIT OR MELON PEEL,350212968,0.084048,379647558,0.286638,488468993,-0.075322,451676290
8,09,"COFFEE, TEA, MATE & SPICES",13702701,0.270346,17407167,0.275173,22197144,0.329267,29505935
9,10,CEREALS,2466361238,-0.475434,1293769326,0.042660,1348961712,-0.482542,698031515


In [20]:
df["total 2018"].sum()

120341425641

In [21]:
# data_2015= test.nlargest(10,"total")
# data_2015_import = data_2015_import.to_csv("data_2015_import.csv", index=False)
# data_2015_export = data_2015_export.to_csv("data_2015_export.csv", index=False)

# data_2016_import = data_2016_import.to_csv("data_2016_import.csv", index=False)
# data_2016_export = data_2016_export.to_csv("data_2016_export.csv", index=False)

# data_2017_import = data_2017_import.to_csv("data_2017_import.csv", index=False)
# data_2017_export = data_2017_export.to_csv("data_2017_export.csv", index=False)

# data_2018_import = data_2018_import.to_csv("data_2018_import.csv", index=False)
# data_2018_export = data_2018_export.to_csv("data_2018_export.csv", index=False)

In [72]:
total_import_export =  pd.DataFrame({"year":["2015","2016","2017","2018"],
                                     "import":[data_2015_import["total"].sum(),data_2016_import["total"].sum(),data_2017_import["total"].sum(),data_2018_import["total"].sum()],
                                     "export":[data_2015_export["total"].sum(),data_2016_export["total"].sum(),data_2017_export["total"].sum(),data_2018_export["total"].sum()]
                                     
})
total_import_export

,year,import,export
0,2015,483201655364,115873365314
1,2016,462542004554,115545507660
2,2017,505469954379,129893586716
3,2018,539503427742,120341425641


In [73]:
total_import_export["deficit"]= total_import_export["import"]-total_import_export["export"]

In [133]:
total_import_export

In [23]:
# data_2015 = data_2015.to_csv("data_2015.csv", index=False)

In [145]:
monthly_import_export = pd.DataFrame({"month":["jan","Feb","Mar","Apr","May","Jun","Jul","Aug","Sep","Oct","Nov","Dec"],
                                     "2017 export":monthly_2017_export["total"],
                                     "2017 import":monthly_2017_import["total"],
                                     "2018 export":monthly_2018_export["total"],
                                     "2018 import":monthly_2018_import["total"]
                                      })
monthly_import_export

,month,2017 export,2017 import,2018 export,2018 import
0,jan,9961097157,41343083151,9835259350,45788040550
1,Feb,9735785625,32804295619,9806092670,39067601646
2,Mar,9719186451,34186942135,12382086329,38256735066
3,Apr,9805657653,37465635527,10268035458,38230010004
4,May,9862177185,41783058622,10610817545,43797369781
5,Jun,9717402236,42289152147,11115623430,44599462713
6,Jul,9979057962,43589168843,10261688039,47096007634
7,Aug,10828339314,45817767860,9294344300,47863906454
8,Sep,10911677353,45429719676,9789081065,50032120606
9,Oct,12963434465,48167652565,9130520323,52232992043


In [134]:
def monthly_data(trade,year):
    month = trade[trade["Period"].str.contains(year)]
    month = month.groupby(["Period"])["Monthly Value"].sum()
    month = pd.DataFrame({"total" : month})
#     data = data.nlargest(10,"total")
    month = month.reset_index()
    
    return month

In [135]:
monthly_2015_export = monthly_data(china_data_export,"2015")
monthly_2015_import = monthly_data(china_data_import,"2015")

monthly_2016_import = monthly_data(china_data_import,"2016")
monthly_2016_export = monthly_data(china_data_export,"2016")

monthly_2017_import = monthly_data(china_data_import,"2017")
monthly_2017_export = monthly_data(china_data_export,"2017")

monthly_2018_import = monthly_data(china_data_import,"2018")
monthly_2018_export = monthly_data(china_data_export,"2018")

In [141]:
monthly_import_export_csv = monthly_import_export.to_csv("monthly_import_export.csv", index=False)

In [24]:
3055129114948

3055129114948